# Yuri Fonseca Lopes - 214450006

# 1. Formalização do problema

Existem várias maneiras de avaliar o desempenho de uma empresa para decidir se é válido comprar ações da mesma. Alguns métodos existentes são:

CAPM (Modelo de precificação de ativos de capital). É um método que analise a relação entre o risco e o retorno que é esperado de um investimento, obtido a partir de princípios de diversificação com pressupostos simplificados.

HME (Hipótese do mercado eficiente). Essa hipótese afirma que os mercados são eficientes, ou seja, um agente não consegue alcançar consistentemente retornos superioers à média do mercado, pois os preços dos ativos refletem toda a informação disponível em um dado momento de tempo.

Análise técnica. É uma busca por padrões recorrentes e previsíveis nos preços do ativo.

Análise Fundamentalista. Utiliza indicadores financeiros, de gestão, do negócio e do risco da empresa, além de índices macroeconômicos na busca de determinar o valor presente descontado dos fluxos de caixa da empresa para chegar ao valor justo de cada ação.

# 2. Mudanças em relação ao TP1

No TP1, eu mapeei os Índices de Governança para valores numéricos e apliquei o K-means nesses novos valores somados. Nesse trabalho não irei realizar essa soma, irei inserir todos os 54 Índices de Governça. Por conta disso, urei usar redução de dimensionalidade para visualização dos clusters.

Fiz uma revisão na literatura (pesquisei assuntos internacionais) que acabou comprovando uma ideia (no meio das milhoes que eu tinha) que eu tive no TP1. Empresas que não pontuam bem no Social e Ambiental (Social and Enviromental) e pontuaram bem em Governança acabam tendo rendimentos maiores que as demais. Fonte: https://www.sciencedirect.com/science/article/pii/S221484502200093X

Esse trabalho citado usou clusterização com o Kmeans++ na bolsa da Turquia. Vou tentar preparar o ambiente para fazer o mesmo aqui com a Bolsa Brasileira. Contudo, terei que adquirir novos dados, pois a base de dados ainda não conta com o rendimento (e outros indicadores de empresas). O trabalho citado usou uma base de dados paga 😒

Essa nova abordagem continua sendo relevante, pois possibilita a identificação de Indices de Governança específicos que indicam bons rendimentos. Com esse novo conhecimento, podemos por exemplo identificar uma empresa que irá realizar um IPO e prever o rendimento dela com base nos possíveis índices que a mesma irá cumprir durante o tempo.


# 3. Análise exploratória de dados (EDA)



In [9]:
# @title Importando os pacotes necessários

import pandas as pd
import requests
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
import plotly.express as px

In [2]:
# @title Carregando o dataset

# URL do arquivo CSV
url = 'https://github.com/Yurovskyy/CDD/raw/main/dataset_CGVN.csv'

# Baixar o arquivo CSV da web
response = requests.get(url)

# Verificar se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Ler o conteúdo do arquivo CSV usando o Pandas
    df = pd.read_csv(url)

# Se a solicitação não for bem-sucedida
else:
    print('Falha ao baixar o arquivo CSV.')

# Obter as dimensões do DataFrame

In [3]:

# @title Exploração inicial do dataset

dimensoes = df.shape
print("Número de entradas(linhas):", dimensoes[0])
print("Número de colunas(variáveis):", dimensoes[1])

# identificar o tipo de cada variável
display(df.dtypes)

# ver as 5 primeiras entradas do conjunto de treino
display(df.head())

Número de entradas(linhas): 115722
Número de colunas(variáveis): 5


Data_Referencia     object
Versao               int64
Nome_Empresarial    object
ID_Item             object
Pratica_Adotada     object
dtype: object

,Data_Referencia,Versao,Nome_Empresarial,ID_Item,Pratica_Adotada
0,2018-12-31,1,BCO BRASIL S.A.,1.1.1,Sim
1,2018-12-31,1,BCO BRASIL S.A.,1.2.1,Sim
2,2018-12-31,1,BCO BRASIL S.A.,1.3.1,Sim
3,2018-12-31,1,BCO BRASIL S.A.,1.3.2,Sim
4,2018-12-31,1,BCO BRASIL S.A.,1.4.1,Não


In [4]:
# @title Valores faltantes
# @markdown Como eu disso, não temos valores faltantes!

# ver a porcentagem valores faltantes
display((df.isnull().sum() / df.shape[0]).sort_values(ascending=False))

Data_Referencia     0.0
Versao              0.0
Nome_Empresarial    0.0
ID_Item             0.0
Pratica_Adotada     0.0
dtype: float64

In [5]:
# @title Descrição dos valores categóricos
# @markdown Temos 499 empresas únicas! (contando com as que se repetem em diferentes intervalos de tempo)

display(df.describe(include=['O']))

,Data_Referencia,Nome_Empresarial,ID_Item,Pratica_Adotada
count,115722,115722,115722,115722
unique,34,499,54,4
top,2022-12-31,GERDAU S.A.,1.1.1,Sim
freq,23004,378,2143,60208


In [6]:
# @title Transformação das variáveis categórias em numéricas
# @markdown Vamos definir PAN como Práticas Adotadas Numéricas

# Criar um dicionário de mapeamento
mapeamento = {
    'Sim': 0.5,
    'Não': -1,
    'Não se Aplica': -0.5,
    'Parcialmente': 0
}

# Usar o método map() para aplicar o mapeamento à coluna
df['PAN'] = df['Pratica_Adotada'].map(mapeamento)

# Exibir o DataFrame com a nova coluna
display(df)

,Data_Referencia,Versao,Nome_Empresarial,ID_Item,Pratica_Adotada,PAN
0,2018-12-31,1,BCO BRASIL S.A.,1.1.1,Sim,0.5
1,2018-12-31,1,BCO BRASIL S.A.,1.2.1,Sim,0.5
2,2018-12-31,1,BCO BRASIL S.A.,1.3.1,Sim,0.5
3,2018-12-31,1,BCO BRASIL S.A.,1.3.2,Sim,0.5
4,2018-12-31,1,BCO BRASIL S.A.,1.4.1,Não,-1.0
...,...,...,...,...,...,...
115717,2024-12-31,1,METALURGICA GERDAU S.A.,5.3.2,Sim,0.5
115718,2024-12-31,1,METALURGICA GERDAU S.A.,5.4.1,Sim,0.5
115719,2024-12-31,1,METALURGICA GERDAU S.A.,5.5.1,Sim,0.5
115720,2024-12-31,1,METALURGICA GERDAU S.A.,5.5.2,Sim,0.5


In [15]:

# X = soma_por_empresa['PAN']
df2 = df
colunas = ['Data_Referencia','Nome_Empresarial','ID_Item','PAN']
X = df2[colunas]
# print(X)

# Transformar data de character para Date
df2['Data_Referencia'] = pd.to_datetime(df2['Data_Referencia'], format='%Y-%m-%d')

# Selecionar colunas relevantes para a clusterização
columns_to_use = ['Data_Referencia', 'Nome_Empresarial', 'ID_Item', 'PAN']

# Filtrar o DataFrame
df2 = df2[columns_to_use]

print(df2)

# Criar transformações para colunas categóricas e numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['PAN']),
        ('cat', OneHotEncoder(sparse_output=False), ['Nome_Empresarial', 'ID_Item'])
    ],
    remainder='passthrough'  # Manter a coluna de data
)

# Aplicar as transformações e obter o array transformado
transformed_data = preprocessor.fit_transform(df2)

# Nomear as colunas resultantes das transformações
one_hot_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(['Nome_Empresarial', 'ID_Item'])
numeric_columns = ['PAN']
date_columns = ['Data_Referencia']

# Concatenar todos os nomes de colunas
all_columns = list(one_hot_columns) + numeric_columns + date_columns

# Criar DataFrame transformado
df_transformed = pd.DataFrame(transformed_data, columns=all_columns)

# Visualizar o DataFrame transformado
print(df_transformed.head())


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)